In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

%matplotlib inline
#%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)


2.9.1


In [2]:
dataset = pd.read_csv('eMalahleniCNN.csv', sep =';', header=0, index_col=0)
values = dataset.values

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
dataset.columns

Index(['pm2.5', 'pm10', 'no2', 'no', 'nox', 'co', 'ws', 'Epm2.5', 'Epm10',
       'Hpm2.5', 'Hpm10', 'Mpm2.5', 'Mpm10', 'Spm2.5', 'Mo3', 'Spm10', 'Sno2',
       'Sno', 'Snox', 'So3', 'Sco'],
      dtype='object')

In [5]:
# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict
# We drop these because we are only interested in predicting for a single variable (pollution).
# If we don't drop, then we will be predicting for all the variables too!
reframed.drop(reframed.columns[[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]], axis=1, inplace=True)
values = reframed.values

In [6]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var18(t-1),var19(t-1),var20(t-1),var21(t-1),var1(t)
1,0.018966,0.016509,0.023121,0.007442,0.015058,0.004532,0.487526,0.022731,0.019660,0.034597,...,0.041381,0.011986,0.148473,0.006100,0.092911,0.004285,0.008461,0.112376,0.002042,0.026136
2,0.026136,0.016882,0.088378,0.006463,0.036362,0.003827,0.455301,0.032196,0.029914,0.034597,...,0.041383,0.006124,0.148473,0.002865,0.092911,0.003727,0.008961,0.112376,0.001518,0.038245
3,0.038245,0.024513,0.121924,0.008580,0.049843,0.024270,0.373181,0.027696,0.023829,0.034597,...,0.041384,0.013940,0.148473,0.006125,0.092911,0.005609,0.027036,0.112376,0.002042,0.015315
4,0.015315,0.009721,0.013445,0.005276,0.009682,0.022155,0.505198,0.108746,0.108117,0.034598,...,0.041386,0.011498,0.148473,0.005003,0.092911,0.004982,0.034272,0.112376,0.001518,0.010145
5,0.010145,0.006823,0.026773,0.006716,0.015604,0.006143,0.279626,0.086102,0.090864,0.034598,...,0.041388,0.008492,0.148473,0.003967,0.092911,0.005121,0.023401,0.112376,0.000995,0.023049


In [7]:
values.shape

(87645, 22)

In [8]:
X = values[:,:-1]

In [9]:
Y = values[:,-1]

In [10]:
X.shape

(87645, 21)

In [11]:
Y.shape

(87645,)

In [12]:

X = X.reshape(X.shape[0],1,1,X.shape[1])

In [13]:
X.shape

(87645, 1, 1, 21)

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

In [15]:
print ('X_train:',X_train.shape)
print ('Y_train:',Y_train.shape)
print ()
print ('X_val:',X_val.shape)
print ('Y_val:',Y_val.shape)
print ()
print ('X_test:',X_test.shape)
print ('Y_test:',Y_test.shape)

X_train: (56092, 1, 1, 21)
Y_train: (56092,)

X_val: (14024, 1, 1, 21)
Y_val: (14024,)

X_test: (17529, 1, 1, 21)
Y_test: (17529,)


# CNN-LSTM EPOCHS

In [16]:
# Define the function to create the CNN-LSTM model
def create_model():
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=256, kernel_size=1, activation='relu'), input_shape=(None, 1, 21)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=256, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=256, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(56, input_shape=(1, 21)))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(1, activation = 'sigmoid'))  
    model.compile(optimizer='adam', loss='mse')
    return model

In [17]:
model = KerasRegressor(model=create_model, loss="mse", batch_size=16, verbose=0)
# define the grid search parameters
epochs = [20, 30, 40, 50, 60]
param_grid = dict( epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

In [18]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.694068 using {'epochs': 40}
0.665761 (0.015991) with: {'epochs': 20}
0.688460 (0.020046) with: {'epochs': 30}
0.694068 (0.006228) with: {'epochs': 40}
0.672719 (0.002726) with: {'epochs': 50}
0.667877 (0.009815) with: {'epochs': 60}


# CNN LSTM FILTERS

In [21]:
# Define the function to create the CNN-LSTM model
def create_model(filters):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1, activation='relu'), input_shape=(None, 1, 21)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=filters, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(56, input_shape=(1, 21)))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(1, activation = 'sigmoid'))  
    model.compile(optimizer='adam', loss='mse')
    return model

In [22]:
model = KerasRegressor(model=create_model, loss="mse", epochs=40, batch_size=16, verbose=0)
# define the grid search parameters
filters = [32, 64, 128, 256]
param_grid = dict(model__filters=filters)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

In [23]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.687946 using {'model__filters': 64}
0.685496 (0.009867) with: {'model__filters': 32}
0.687946 (0.005790) with: {'model__filters': 64}
0.677468 (0.013312) with: {'model__filters': 128}
0.673452 (0.019163) with: {'model__filters': 256}


# CNN LSTM NEURONS

In [24]:
# Define the function to create the CNN-LSTM model
def create_model(neurons):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, 1, 21)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(neurons, input_shape=(1, 21)))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation = 'sigmoid'))  
    model.compile(optimizer='adam', loss='mse')
    return model

In [25]:
model = KerasRegressor(model=create_model, loss="mse", epochs=40, batch_size=16, verbose=0)
# define the grid search parameters
neurons = [28, 56, 112, 224, 448]
param_grid = dict(model__neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

In [26]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.679812 using {'model__neurons': 56}
0.678345 (0.012911) with: {'model__neurons': 28}
0.679812 (0.013951) with: {'model__neurons': 56}
0.223899 (0.670912) with: {'model__neurons': 112}
-0.276707 (0.675447) with: {'model__neurons': 224}
-0.758792 (0.024754) with: {'model__neurons': 448}


# CNN LSTM BATCH SIZE

In [27]:
# Define the function to create the CNN-LSTM model
def create_model():
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, 1, 21)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu')))
    model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(56, input_shape=(1, 21)))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(56, activation='relu'))
    model.add(Dense(1, activation = 'sigmoid'))  
    model.compile(optimizer='adam', loss='mse')
    return model

In [28]:
model = KerasRegressor(model=create_model, loss="mse", epochs=40, verbose=0)
# define the grid search parameters
batch_size = [16, 32, 64,128, 256]
param_grid = dict( batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

In [29]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.689898 using {'batch_size': 256}
0.684880 (0.004156) with: {'batch_size': 16}
0.688970 (0.006966) with: {'batch_size': 32}
0.684239 (0.015562) with: {'batch_size': 64}
0.680238 (0.007914) with: {'batch_size': 128}
0.689898 (0.008497) with: {'batch_size': 256}


In [ ]:
def create_model( hidden_layers = 1):
  # Initialize the constructor
  model = Sequential()
  # Add an input layer
  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, 1, 13)))
  model.add(TimeDistributed(MaxPooling1D(pool_size=1)))

  for i in range(hidden_layers):
      # Add one hidden layer
      model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu')))
      model.add(TimeDistributed(MaxPooling1D(pool_size=1)))
      model.add(TimeDistributed(Flatten()))
      model.add(LSTM(28, input_shape=(1, 13)))
      model.add(Dense(28, activation='relu'))
        
  # Add an output layer 
  model.add(Dense(1, activation = 'sigmoid'))
  #compile model
  model.compile(loss='mse', optimizer='adam')
  return model

In [ ]:
model = KerasRegressor(model=create_model, loss="mse", hidden_layers=1, epochs=40, batch_size=32, verbose=0)
# define the grid search parameters
hidden_layers = [1, 2, 3, 4]
param_grid = dict(hidden_layers=hidden_layers)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))